In [1]:
from psycopg2.extras import RealDictCursor
import psycopg2
import pandas as pd

In [2]:
def create_connection():
    return psycopg2.connect("dbname=twitter user=postgres password=postgres host=localhost port=5433")

In [3]:
create_table = """
CREATE TABLE sentiment_ratio_by_weeks (
    id SERIAL PRIMARY KEY,
    theory_id INTEGER NOT NULL,
    tweet_count INTEGER DEFAULT 0,
    tweet_extreme_count INTEGER DEFAULT 0,
    tweet_neutral_count INTEGER DEFAULT 0,
    ratio DECIMAL DEFAULT 0,
    week INTEGER,
    year INTEGER,
    FOREIGN KEY (theory_id) REFERENCES theories(id) ON UPDATE CASCADE ON DELETE RESTRICT
)
"""

conn = create_connection();
cur = conn.cursor()
cur.execute(create_table)
conn.commit()
conn.close()

In [5]:
select = """
INSERT INTO sentiment_ratio_by_weeks (theory_id, year, week, tweet_extreme_count, tweet_neutral_count, tweet_count, ratio)
SELECT sub.*, CASE WHEN sub.neutral = 0 THEN NULL ELSE sub.extreme::DECIMAL / sub.neutral::DECIMAL END
FROM
(SELECT tt.theory_id, DATE_PART('year', t.happened_at) as year, DATE_PART('week', t.happened_at) as week, COUNT(CASE WHEN t.compound < -0.5 OR t.compound > 0.5 THEN t.compound END) as extreme, COUNT(CASE WHEN t.compound >= -0.5 AND t.compound <= 0.5 THEN t.compound END) as neutral, count(tt.theory_id) as t_count
FROM tweets t
JOIN tweets_theories tt ON tt.tweet_id = t.id
GROUP BY tt.theory_id, year, week) as sub
"""

conn = create_connection();
cur = conn.cursor()
cur.execute(select)
conn.commit()
conn.close()